In [1]:
# Api Key
from dotenv import load_dotenv
import openai
import os

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")
original_create = openai.chat.completions.create

In [2]:
import pprint

def pretty_print_file(file_name):
    with open(file_name, 'r') as f:
        pp = pprint.PrettyPrinter(indent=4, width = 150)
        pp.pprint(f.read())

In [3]:
# Set Once and every call will create an AIConfig
import openai
from aiconfig.ChatCompletion import create_and_save_to_config
openai.chat.completions.create = create_and_save_to_config()

# Basic Execution
completion_params = {
            "model": "gpt-3.5-turbo",
            "top_p": 1,
            "max_tokens": 3000,
            "temperature": 1,
            "stream": False,
            "messages": [
                {
                    "content": "Hi",
                    "role": "user",
                }
            ],
        }

response = openai.chat.completions.create(**completion_params) # Creates a config saved to default path `aiconfig.json`
print("Chat Completion Response: ")
pprint.pprint(response)

print("\nGenerated AIConfig Json: ")
pretty_print_file('aiconfig.json')

/opt/homebrew/Caskroom/miniconda/base/envs/aiconfig/lib/python3.10/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_parsers" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/aiconfig/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Chat Completion Response: 
ChatCompletion(id='chatcmpl-8PgfXpinkNKYLUUhswWDHm8diAP7c', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1701132911, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=9, prompt_tokens=8, total_tokens=17))

Generated AIConfig Json: 
('{\n'
 '  "name": "",\n'
 '  "schema_version": "latest",\n'
 '  "metadata": {\n'
 '    "parameters": {},\n'
 '    "models": {}\n'
 '  },\n'
 '  "description": "",\n'
 '  "prompts": [\n'
 '    {\n'
 '      "name": "prompt_0",\n'
 '      "input": "Hi",\n'
 '      "metadata": {\n'
 '        "model": {\n'
 '          "name": "gpt-3.5-turbo",\n'
 '          "settings": {\n'
 '            "model": "gpt-3.5-turbo",\n'
 '            "top_p": 1,\n'
 '            "max_tokens": 3000,\n'
 '            "temperature": 1,\n'
 '         

In [4]:
# Set Once and every call will create an AIConfig
import openai
from aiconfig.ChatCompletion import create_and_save_to_config
# Set file path
aiconfig_file_path = "my-second-aiconfig.json"
openai.chat.completions.create  = create_and_save_to_config(config_file_path=aiconfig_file_path)

# Basic Streaming Execution
completion_params = {
            "model": "gpt-3.5-turbo",
            "top_p": 1,
            "max_tokens": 3000,
            "temperature": 1,
            "stream": True,
            "messages": [
                {
                    "content": "Tell me a riveting story",
                    "role": "user",
                }
            ],
        }

response = openai.chat.completions.create(**completion_params) # Creates a config saved to aiconfig_file_path = "my-second-aiconfig.json"
print("Chat Completion Streaming Response: ")
for iteration in response:
    chunk = iteration.get("choices",[{}])[0].get('delta',{}).get('content','')
    print(chunk, end = '')


print("\nGenerated AIConfig Json: ")
pretty_print_file('aiconfig.json')
from openai import *

Chat Completion Streaming Response: 
Once upon a time, in a quaint little village nestled in the heart of a magical forest, lived a young boy named Oliver. Oliver was an adventurous and curious soul, always seeking a new and thrilling experience. One day, while exploring the woods, he stumbled upon an ancient, weathered map tucked inside a hollowed-out tree trunk.

Intrigued by the mysterious markings on the map, Oliver decided to embark on a quest to uncover the treasure it promised. With his loyal companion, a mischievous fox named Lily, by his side, they followed the map's directions, leading them deep into the heart of the forest.

As they ventured further, the forest became thicker, the air heavy with enchantment. Suddenly, they stumbled upon an immense stone door adorned with intricate carvings and symbols. Eager to unlock the secrets beyond, Oliver examined the map once more, looking for any clues.

The map indicated that to gain entry, they needed to collect three enchanted key

In [5]:
# Set Once and every call will create an AIConfig
from aiconfig.Config import AIConfigRuntime
import openai
from aiconfig.ChatCompletion import create_and_save_to_config
# Set aiconfig
aiconfig = AIConfigRuntime.load("my-second-aiconfig.json") # load the second aiconfig we just created and use it
openai.chat.completions.create = create_and_save_to_config(aiconfig=aiconfig)

# Compounded Execution
completion_params = {
            "model": "gpt-3.5-turbo",
            "top_p": 1,
            "max_tokens": 3000,
            "temperature": 1,
            "messages": [
                {
                    "content": "Tell me a joke about apples",
                    "role": "user",
                }
            ],
        }

response = openai.chat.completions.create(**completion_params) # Config was previously loaded from 'my-second-aiconfig.json. New Prompt gets added to the AIConfig Object and also saved to the same file

completion_params = {
            "model": "gpt-3.5-turbo",
            "top_p": 1,
            "max_tokens": 3000,
            "temperature": 1,
            "messages": [
                {
                    "content": "Tell me a joke about apples",
                    "role": "user",
                },
                 {
        "role": "assistant",
        "content": "Why did the apple go to school?\n\nBecause it wanted to be a \"smarty-pie\"!"
      },                                
      {
                    "content": "Tell this joke in a shakespearean rhetoric",
                    "role": "user",
                }
            ],
        }

response = openai.chat.completions.create(**completion_params) # Config is updated with the second prompt keeping the history intact



print("\nGenerated AIConfig Json: ")
pretty_print_file('my-second-aiconfig.json') # open the config yourself and check it out!


Generated AIConfig Json: 
('{\n'
 '  "name": "",\n'
 '  "schema_version": "latest",\n'
 '  "metadata": {\n'
 '    "parameters": {},\n'
 '    "models": {}\n'
 '  },\n'
 '  "description": "",\n'
 '  "prompts": [\n'
 '    {\n'
 '      "name": "prompt_0",\n'
 '      "input": "Tell me a riveting story",\n'
 '      "metadata": {\n'
 '        "model": {\n'
 '          "name": "gpt-3.5-turbo",\n'
 '          "settings": {\n'
 '            "model": "gpt-3.5-turbo",\n'
 '            "top_p": 1,\n'
 '            "max_tokens": 3000,\n'
 '            "temperature": 1,\n'
 '            "stream": true\n'
 '          }\n'
 '        },\n'
 '        "parameters": {},\n'
 '        "remember_chat_context": true\n'
 '      },\n'
 '      "outputs": [\n'
 '        {\n'
 '          "output_type": "execute_result",\n'
 '          "execution_count": 0,\n'
 '          "data": {\n'
 '            "content": "Once upon a time, in a quaint little village nestled in the heart of a magical forest, lived a young boy n

In [6]:
# Function Call Capture
# Set Once and every call will create an AIConfig
from aiconfig.Config import AIConfigRuntime
import openai
from aiconfig.ChatCompletion import create_and_save_to_config
# Set aiconfig
openai.chat.completions.create = create_and_save_to_config(config_file_path='function_call.json')

def get_current_weather(location, unit):
    return { "temperature": 22, "unit": "celsius", "description": "Sunny" }

completion_params = {
    "model": "gpt-3.5-turbo-0613",
    "messages": [{"role": "user", "content": "What is the weather like in Boston?"}],
    "functions": [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ],
}

response = openai.chat.completions.create(**completion_params) 

function_call_response = get_current_weather(location="Boston", unit="celsius")
print(response)

completion_params = {
  "model": "gpt-3.5-turbo-0613",
  "messages": [
    {"role": "user", "content": "What is the weather like in Boston?"},
    {"role": "assistant", "content": 'null', "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }},
    {"role": "function", "name": "get_current_weather", "content": str(function_call_response)}

  ],
  "functions": [
    {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"]
          }
        },
        "required": ["location"]
      }
    }
  ]
}

response = openai.chat.completions.create(**completion_params) 
print(response)

ChatCompletion(id='chatcmpl-8PgfoEnPJi7X0azDcNH2ZhIVrgdrW', choices=[Choice(finish_reason='function_call', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None))], created=1701132928, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=18, prompt_tokens=82, total_tokens=100))
ChatCompletion(id='chatcmpl-8PgfpSm2wx9i2XOM351FMWwjtWzXN', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='The weather in Boston is currently sunny with a temperature of 22 degrees Celsius.', role='assistant', function_call=None, tool_calls=None))], created=1701132929, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=17, prompt_tokens=134, total_tokens=151))


In [7]:
import openai

# Instantiate a client
from aiconfig.ChatCompletion import create_and_save_to_config

client = openai.Client()
client.chat.completions.create = create_and_save_to_config()

completion_params = {
            "model": "gpt-3.5-turbo",
            "top_p": 1,
            "max_tokens": 3000,
            "temperature": 1,
            "stream": False,
            "messages": [
                {
                    "content": "Hi 2",
                    "role": "user",
                }
            ],
        }
response = client.chat.completions.create(**completion_params)


In [1]:
from aiconfig import AIConfigRuntime
from aiconfig import InferenceOptions
import openai

config = AIConfigRuntime.load("aiconfig.json")
inference_options = InferenceOptions()
await config.run("prompt_0", options = inference_options)

/opt/homebrew/Caskroom/miniconda/base/envs/aiconfig/lib/python3.10/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_parsers" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/aiconfig/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


response=[ChatCompletionChunk(id='chatcmpl-8PgmnT5R8rL3MYcPvRypNwylzxwZj', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, role='assistant', tool_calls=None), finish_reason=None, index=0)], created=1701133361, model='gpt-3.5-turbo-0613', object='chat.completion.chunk', system_fingerprint=None), ChatCompletionChunk(id='chatcmpl-8PgmnT5R8rL3MYcPvRypNwylzxwZj', choices=[Choice(delta=ChoiceDelta(content='Hello', function_call=None, role=None, tool_calls=None), finish_reason=None, index=0)], created=1701133361, model='gpt-3.5-turbo-0613', object='chat.completion.chunk', system_fingerprint=None), ChatCompletionChunk(id='chatcmpl-8PgmnT5R8rL3MYcPvRypNwylzxwZj', choices=[Choice(delta=ChoiceDelta(content='!', function_call=None, role=None, tool_calls=None), finish_reason=None, index=0)], created=1701133361, model='gpt-3.5-turbo-0613', object='chat.completion.chunk', system_fingerprint=None), ChatCompletionChunk(id='chatcmpl-8PgmnT5R8rL3MYcPvRypNwylzxwZj', choices=[Choice(delta

[ExecuteResult(output_type='execute_result', execution_count=0, data={'content': 'Hello! How can I assist you today?', 'role': 'assistant'}, mime_type=None, metadata={'finish_reason': 'stop'})]